In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# 1. 데이터 로드
data = pd.read_csv('abalone.csv')

# 2. id 컬럼 드롭
data.drop(columns=['id'], inplace=True)

# 3. 범주형 변수를 원핫 인코딩
data = pd.get_dummies(data, columns=['Sex'], drop_first=True)

# 4. NaN 값 확인 및 처리
data.isnull().sum()  # 결측치 확인
data.fillna(0, inplace=True)  # 결측치가 있을 경우 0으로 대체

# 5. 특성과 타깃 분리
X = data.drop(columns=['Rings'])
y = data['Rings']

In [5]:

# 6. 데이터 타입 확인
X = X.astype(np.float32)  # 데이터 타입 변환 // 근데 왜 계속 object가 생기는 건지 확인해야할 듯
y = y.astype(np.float32)


In [6]:
# 7. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 8. LSTM 모델 입력 형태로 차원 조정
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))


In [7]:
# 9. LSTM 모델 정의
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))

# 10. 모델 컴파일
model.compile(optimizer='adam', loss='mse')

C:\Users\tyra\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
# 11. 모델 학습
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# 12. 모델 평가
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

Epoch 1/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 102.1798 - val_loss: 70.3599
Epoch 2/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 52.6055 - val_loss: 15.0958
Epoch 3/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14.9372 - val_loss: 12.2148
Epoch 4/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 12.5011 - val_loss: 11.5141
Epoch 5/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 12.0633 - val_loss: 10.8767
Epoch 6/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11.2668 - val_loss: 10.3227
Epoch 7/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10.6655 - val_loss: 9.6649
Epoch 8/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10.5903 - val_loss: 8.9863
Epoch 9/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9.6120 - val_loss: 8.3628
Epoch 10/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9.0456 - val_loss: 7.7194
Epoch 11/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7.9418 - val_loss: 7.2096
Epoch 12/100
105/105 ━━━━━

In [10]:
from sklearn.metrics import r2_score

y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R² Score: {r2}')


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
R² Score: 0.548499739984797


In [11]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 4.887584686279297


## 기본적인 것만 했을 때, 중간 정도의 성능을 보여준다. CNN보다는 나은 것 같다.
## 그래도 뭔가 성능을  더 높여보도록 하자.

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

# 1. 데이터 로드
data = pd.read_csv('abalone.csv')

# 2. id 컬럼 드롭
data.drop(columns=['id'], inplace=True)

# 3. 범주형 변수를 원핫 인코딩
data = pd.get_dummies(data, columns=['Sex'], drop_first=True)

# 4. NaN 값 확인 및 처리
data.fillna(0, inplace=True)  # 결측치가 있을 경우 0으로 대체

# 5. 특성과 타깃 분리
X = data.drop(columns=['Rings'])
y = data['Rings']

# 6. 데이터 타입 확인
X = X.astype(np.float32)  # 데이터 타입 변환
y = y.astype(np.float32)

# 7. 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 8. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 9. LSTM 모델 입력 형태로 차원 조정
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# 10. LSTM 모델 정의
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

# 11. 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 12. 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# 13. 모델 학습
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# 14. 모델 평가
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')


Epoch 1/100


C:\Users\tyra\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 102.1154 - val_loss: 54.5583
Epoch 2/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 48.8309 - val_loss: 16.3139
Epoch 3/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15.3108 - val_loss: 5.7092
Epoch 4/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.9517 - val_loss: 5.6184
Epoch 5/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.6898 - val_loss: 5.1646
Epoch 6/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.1061 - val_loss: 5.0201
Epoch 7/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.1438 - val_loss: 4.9645
Epoch 8/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.1939 - val_loss: 4.9762
Epoch 9/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.3023 - val_loss: 4.8010
Epoch 10/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.5129 - val_loss: 4.7213
Epoch 11/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.7316 - val_loss: 4.8658
Epoch 12/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2m

1. 데이터 스케일링: StandardScaler를 사용하여 특성 값을 표준화합니다.
2. 모델 구조: 두 개의 LSTM 층과 Dropout 층을 추가하여 모델의 복잡성을 높였습니다.
3. 조기 종료(EarlyStopping): 검증 손실이 개선되지 않으면 학습을 조기에 종료합니다.

### 위의 내용을 사용해 test loss를 0.2 낮추었다. 조금이라도 개선되었다.

#### 하지만 LSTM이 무엇인가? 맥락 다시 말해 시간을 고려하는 것 아니겠는가?
#### 솔직히 time step에 변화를 준다고 획기적인 변화가 있을 것 같지는 않지만 time step도 만져보자

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

# 1. 데이터 로드
data = pd.read_csv('abalone.csv')

# 2. id 컬럼 드롭
data.drop(columns=['id'], inplace=True)

# 3. 범주형 변수를 원핫 인코딩
data = pd.get_dummies(data, columns=['Sex'], drop_first=True)

# 4. NaN 값 확인 및 처리
data.fillna(0, inplace=True)  # 결측치가 있을 경우 0으로 대체

# 5. 특성과 타깃 분리
X = data.drop(columns=['Rings'])
y = data['Rings']

# 6. 데이터 타입 확인
X = X.astype(np.float32)  # 데이터 타입 변환
y = y.astype(np.float32)

# 7. 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 8. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 9. LSTM 모델 입력 형태로 차원 조정 (timesteps를 3으로 설정)
timesteps = 3
X_train_seq = []
y_train_seq = []
X_test_seq = []
y_test_seq = []

# 훈련 데이터 시퀀스 생성
for i in range(len(X_train) - timesteps):
    X_train_seq.append(X_train[i:i + timesteps])
    y_train_seq.append(y_train.iloc[i + timesteps])  # 다음 값 예측

# 테스트 데이터 시퀀스 생성
for i in range(len(X_test) - timesteps):
    X_test_seq.append(X_test[i:i + timesteps])
    y_test_seq.append(y_test.iloc[i + timesteps])  # 다음 값 예측

X_train_seq = np.array(X_train_seq)
y_train_seq = np.array(y_train_seq)
X_test_seq = np.array(X_test_seq)
y_test_seq = np.array(y_test_seq)

# 10. LSTM 모델 정의
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(timesteps, X_train_seq.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

# 11. 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 12. 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# 13. 모델 학습
model.fit(X_train_seq, y_train_seq, epochs=100, batch_size=32, validation_data=(X_test_seq, y_test_seq), callbacks=[early_stopping])

# 14. 모델 평가
loss = model.evaluate(X_test_seq, y_test_seq)
print(f'Test Loss: {loss}')


Epoch 1/100


C:\Users\tyra\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 84.2305 - val_loss: 17.9607
Epoch 2/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 16.7973 - val_loss: 11.2955
Epoch 3/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.6408 - val_loss: 11.4709
Epoch 4/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.6341 - val_loss: 11.1264
Epoch 5/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11.7285 - val_loss: 11.0332
Epoch 6/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.0567 - val_loss: 11.0240
Epoch 7/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11.6876 - val_loss: 11.2993
Epoch 8/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11.3800 - val_loss: 11.3941
Epoch 9/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11.8858 - val_loss: 11.0485
Epoch 10/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11.4422 - val_loss: 10.9440
Epoch 11/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11.2794 - val_loss: 10.9516
Epoch 12/100
105/105 ━━━━━━━━━━

### 오히려 맥락이 쓸모없는 데이터라 그런지 timesteps을 3으로 설정하였더니 더 큰 loss가 발생하였다.